In [11]:
import pandas as pd
import os
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input
from sklearn.preprocessing import RobustScaler, LabelEncoder
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
import keras.backend as K

entorno = 'local'  # Elegir "VM" o "local" para correr en entorno local
cantidad_productos_cliente_mayor_ventas = 50
nombre_experimento = 'LSTM_producto_v3'
ventana_input = 3
lags = 3
ventana_output = 2

# Configurar entorno
if entorno == 'VM':
    carpeta_datasets = os.path.expanduser('~/buckets/b1/datasets')
    carpeta_exp_base = os.path.expanduser('~/buckets/b1/exp')
elif entorno == 'local':
    carpeta_datasets = 'C:\\Users\\alope\\Desktop\\Trámites\\Maestria Data Science - Universidad Austral\\Laboratorio de implementación 3\\Datos'
    carpeta_exp_base = 'C:\\Users\\alope\\Desktop\\Trámites\\Maestria Data Science - Universidad Austral\\Laboratorio de implementación 3\\Resultados'
else:
    raise Exception("Entorno especificado incorrectamente")

carpeta_exp = os.path.join(carpeta_exp_base, nombre_experimento)
if not os.path.exists(carpeta_exp):
    os.makedirs(carpeta_exp)
    
dataset_completo = pd.read_csv(os.path.join(carpeta_datasets, 'df_producto_cliente_completo.csv'))


dataset_completo.head()

,Timestamp,customer_id,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,cat1,cat2,cat3,...,sku_size,descripcion,mes,quarter,fin_quarter,edad_producto,ventas_cat1,ventas_cat2,ventas_cat3,ventas_familia_producto
0,2017-01-01,10234,20524,0.0,2.0,0.05300,0.05300,HC,VAJILLA,Cristalino,...,500.0,Abrillantador,1,1,0,0,14.31686,4.96628,3.03194,0.25684
1,2017-02-01,10234,20524,NaN,0.0,0.00000,0.00000,HC,VAJILLA,Cristalino,...,500.0,Abrillantador,2,1,0,1,2.14290,0.10339,0.00000,0.00000
2,2017-03-01,10234,20524,0.0,1.0,0.01514,0.01514,HC,VAJILLA,Cristalino,...,500.0,Abrillantador,3,1,1,2,8.59237,2.23835,1.52777,0.04699
3,2017-04-01,10234,20524,NaN,0.0,0.00000,0.00000,HC,VAJILLA,Cristalino,...,500.0,Abrillantador,4,2,0,3,9.18260,4.47157,2.35257,0.00000
4,2017-05-01,10234,20524,NaN,0.0,0.00000,0.00000,HC,VAJILLA,Cristalino,...,500.0,Abrillantador,5,2,0,4,7.79714,0.50130,0.09348,0.00000


In [12]:
ventas_producto_mes = dataset_completo.groupby(['Timestamp', 'product_id', 'cat1', 'cat2', 'cat3', 'brand', 'descripcion']).agg({
    'tn': 'sum',
    'edad_producto': 'first'
})
ventas_producto_mes = ventas_producto_mes.reset_index()

#Generar ventas por categoria y agregar al dataset
ventas_cat1 = ventas_producto_mes.groupby(['Timestamp', 'cat1'])['tn'].sum().reset_index().rename(columns={'tn': 'ventas_cat1'})
ventas_cat2 = ventas_producto_mes.groupby(['Timestamp', 'cat1', 'cat2'])['tn'].sum().reset_index().rename(columns={'tn': 'ventas_cat2'})
ventas_cat3 = ventas_producto_mes.groupby(['Timestamp', 'cat1', 'cat2', 'cat3'])['tn'].sum().reset_index().rename(columns={'tn': 'ventas_cat3'})
ventas_familia_productos = ventas_producto_mes.groupby(['Timestamp', 'cat1', 'cat2', 'cat3', 'brand', 'descripcion'])['tn'].sum().reset_index().rename(columns={'tn': 'ventas_familia_producto'})

ventas_producto_mes = ventas_producto_mes.merge(ventas_cat1, how='left', on=['Timestamp','cat1'])
ventas_producto_mes = ventas_producto_mes.merge(ventas_cat2, how='left', on=['Timestamp', 'cat1', 'cat2'])
ventas_producto_mes = ventas_producto_mes.merge(ventas_cat3, how='left', on=['Timestamp', 'cat1', 'cat2', 'cat3'])
ventas_producto_mes = ventas_producto_mes.merge(ventas_familia_productos, how='left', on=['Timestamp', 'cat1', 'cat2', 'cat3', 'brand', 'descripcion'])

ventas_producto_mes.drop(columns=['cat1', 'cat2', 'cat3', 'brand', 'descripcion'], inplace = True)


In [13]:
# Crear lags y rellenar NA
def crear_lags(df, lags):
    for lag in range(ventana_input, ventana_input+ lags + 1):
        df[f'tn_lag_{lag}'] = df['tn'].shift(lag)
        df[f'ventas_cat1_lag_{lag}'] = df['ventas_cat1'].shift(lag)
        df[f'ventas_cat2_lag_{lag}'] = df['ventas_cat2'].shift(lag)
        df[f'ventas_cat3_lag_{lag}'] = df['ventas_cat3'].shift(lag)
        df[f'ventas_familia_producto_lag_{lag}'] = df['ventas_familia_producto'].shift(lag)
    df.dropna(inplace = True)
    return df

def crear_dataset_supervisado(array, input_length, output_length):
    # Inicialización
    X, Y = [], []    # Listados que contendrán los datos de entrada y salida del modelo
    shape = array.shape
    if len(shape)==1: # Si tenemos sólo una serie (univariado)
        fils, cols = array.shape[0], 1
        array = array.reshape(fils,cols)
    else: # Multivariado
        fils, cols = array.shape
    # Generar los arreglos (utilizando ventanas deslizantes de longitud input_length)
    for i in range(fils-input_length-output_length + 1):
        X.append(array[i:i+input_length,0:cols])
        Y.append(array[i+input_length:i+input_length+output_length,-1].reshape(output_length,1))

    # Convertir listas a arreglos de NumPy
    X = np.array(X)
    Y = np.array(Y)

    return X, Y


def error_pred_sobre_venta(y_true, y_pred):
    # Seleccionar solo el segundo valor de la ventana de salida
    y_true_second = y_true[:, 1]
    y_pred_second = y_pred[:, 1]
    
    abs_diff = K.abs(y_true_second - y_pred_second)
    sum_abs_diff = K.sum(abs_diff)
    sum_y_true = K.sum(y_true_second)
    return sum_abs_diff / sum_y_true

# Definir función para crear y entrenar el modelo
def crear_modelo(ventana_input, ventana_output, lstm_units=[128,64, 32], dropout_rate=0.25):
    model = Sequential()
    model.add(Input(shape=(ventana_input, X.shape[2])))
    for units in lstm_units[0:-1]:
        model.add(LSTM(units, return_sequences = True))
        model.add(Dropout(dropout_rate))
    model.add(LSTM(units, return_sequences = False))
    model.add(Dropout(dropout_rate))
    model.add(Dense(ventana_output))
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=[error_pred_sobre_venta])
    return model


In [14]:
EPOCHS = 100
BATCH_SIZE = 32

productos_con_prediccion = pd.DataFrame()

# Entrenar y predecir
for producto in ventas_producto_mes['product_id'].unique():
    ventas_mes_por_producto = ventas_producto_mes[(ventas_producto_mes['product_id'] == producto)].copy()
    timestamp_primera_compra = min(ventas_mes_por_producto['Timestamp'])
    if timestamp_primera_compra <= '2018-12-01':
        ventas_mes_por_producto.drop(columns=['product_id'], inplace=True)
        ventas_mes_por_producto.set_index('Timestamp', inplace = True)

        dict_scalers = {}
        for columna in ['tn', 'edad_producto', 'ventas_cat1', 'ventas_cat2', 'ventas_cat3', 'ventas_familia_producto']:
            scaler = RobustScaler()
            ventas_mes_por_producto[columna] = scaler.fit_transform(ventas_mes_por_producto[columna].values.reshape(-1,1))
            dict_scalers[columna] = scaler

        ventas_mes_por_producto = crear_lags(ventas_mes_por_producto, lags)
        X, Y = crear_dataset_supervisado(np.array(ventas_mes_por_producto), ventana_input, ventana_output)
        
        early_stop = EarlyStopping(monitor='loss', patience=20)
        reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.2, patience=10, min_lr=1e-6)
        
        model = crear_modelo(ventana_input, ventana_output)

        history = model.fit(X, Y, epochs=EPOCHS, batch_size=BATCH_SIZE, callbacks=[early_stop, reduce_lr], verbose=0)
        
        error_modelo = history.history['error_pred_sobre_venta']
        
        print(f'Error Producto {producto}: {error_modelo[-1]}')
        y_pred_s = model.predict(X[-1].reshape(1,X.shape[1],X.shape[2]), verbose=0)
        y_pred = dict_scalers['tn'].inverse_transform(y_pred_s)
        prediccion_mes_2 = max(0,y_pred.flatten()[1])
        
    else:
        prediccion_mes_2 = ventas_mes_por_producto['tn'].mean()
    nueva_prediccion = pd.DataFrame({'product_id': [producto], 'prediccion': [prediccion_mes_2]})
    productos_con_prediccion = pd.concat([productos_con_prediccion, nueva_prediccion], axis = 0)
    productos_con_prediccion.to_csv(os.path.join(carpeta_exp,nombre_experimento + 'prediccion.csv'), index = False)

Error Producto 20001: -38.66535186767578
Error Producto 20002: 2.0691580772399902
Error Producto 20003: 0.5341546535491943
Error Producto 20004: 0.4411853849887848
Error Producto 20005: 0.4855020046234131
Error Producto 20006: 0.5477057099342346
Error Producto 20007: 0.8151127099990845
Error Producto 20008: 1.3141069412231445
Error Producto 20009: -0.636580228805542
Error Producto 20010: 1.9716955423355103
Error Producto 20011: 0.5325804948806763
Error Producto 20012: 1.250859022140503
Error Producto 20013: 1.8807348012924194
Error Producto 20014: 0.728859007358551
Error Producto 20015: -3.0912787914276123
Error Producto 20016: 1.5940052270889282
Error Producto 20017: 0.5542606115341187
Error Producto 20018: 0.5115575790405273
Error Producto 20019: 0.46799248456954956
Error Producto 20020: 0.7261643409729004
Error Producto 20021: 0.7991684675216675
Error Producto 20022: 2.0841238498687744
Error Producto 20023: 0.8879711031913757
Error Producto 20024: 0.7974767088890076
Error Producto 2